In [48]:
import pandas as pd
import numpy as np

In [49]:
data = pd.read_csv("./mushrooms.csv")

In [50]:
data.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [51]:
data.shape

(8124, 23)

In [52]:
X,y = data.drop(['type'],axis = 1),data['type'].values
feature_names = data.columns[1:]

In [53]:
feature_names

Index(['cap_shape', 'cap_surface', 'cap_color', 'bruises', 'odor',
       'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color',
       'stalk_shape', 'stalk_root', 'stalk_surface_above_ring',
       'stalk_surface_below_ring', 'stalk_color_above_ring',
       'stalk_color_below_ring', 'veil_type', 'veil_color', 'ring_number',
       'ring_type', 'spore_print_color', 'population', 'habitat'],
      dtype='object')

In [54]:
def entropy(y):
    classes,counts = np.unique(y,return_counts=True)
    prob = counts/len(y)

    return -1*np.sum(prob*np.log2(prob))

In [55]:
entropy([1,0,1,0])

1.0

In [56]:
l = np.array([1,0,1,0,1,1,1,2,2,2,0])
l == 1

array([ True, False,  True, False,  True,  True,  True, False, False,
       False, False])

In [91]:
class DecisionTree:
    def __init__(self,max_depth = 10,min_split = 2):
        self.max_depth = max_depth
        self.min_split = min_split
        
    def fit(self,X,y):
        self.labels = np.unique(y)
        self.root = self.constructNode(X,y,0)
        # return self.root

    def traverse_predict(self,t,node):
        if node['isLeaf'] == True:
            return node['Predictions']
        else:
            return self.traverse_predict(t,node['children'][t[node['column']]])

    def predict(self,t):
        predictions = []
        for i in range(len(t)):
            pred = self.traverse_predict(t.iloc[i],self.root)
            predictions.append(self.labels[np.argmax(pred)])
            
        return np.array(predictions)
    
    def constructNode(self,X,y,cur_depth):
        if len(X) == 0 or cur_depth > self.max_depth:
            return None

        if len(X) >= self.min_split:
            parentEntropy = self.entropy(y)
            weightedChildrenEntropy = np.zeros((X.shape[1],))
    
            for feat in range(X.shape[1]):
                feat_values = X.iloc[:,feat].unique()
                for feat_value in feat_values:
                    weightedChildrenEntropy[feat] += (X.iloc[:,feat] == feat_value).mean()*self.entropy(y[X.iloc[:,feat] == feat_value])
    
            feat = np.argmax(parentEntropy - weightedChildrenEntropy)
            
            node = {'column':X.columns[feat], 
                    'children':{},
                    'isLeaf':False
                   }
                    
            feat_values = X.iloc[:,feat].unique()
            for feat_value in feat_values:
                X_split = X.drop([X.columns[feat]],axis=1)[X.iloc[:,feat] == feat_value]
                y_split = y[X.iloc[:,feat] == feat_value]
                child_node = self.constructNode(X_split,y_split,cur_depth+1)
                if child_node == None:
                    node['isLeaf'] = True
                    break
                node['children'][feat_value] = child_node

        else:
            node = {'column':None, 
                    'children':{},
                    'isLeaf':True
                   }

        if node['isLeaf']:
            node['Predictions'] = self.predictions(y)

        return node

    def predictions(self,y):
        prob = np.zeros(self.labels.shape,dtype=np.float32)
        for i,cls in enumerate(self.labels):
            prob[i] = (y==cls).mean()

        return prob
            

    def entropy(self,y):
        classes,counts = np.unique(y,return_counts=True)
        prob = counts/len(y)
    
        return -1*np.sum(prob*np.log2(prob))

In [92]:
dt = DecisionTree(min_split=100)
dt.fit(X,y)
# dt.root

In [93]:
ypred = dt.predict(X)

In [94]:
(y==ypred).mean()

0.9970457902511078

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [97]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size = 0.8)

In [103]:
dt = DecisionTree(max_depth=5,min_split=200)
# dt = DecisionTreeClassifier(max_depth=5,min_samples_split=200)
dt.fit(x_train,y_train)

In [104]:
accuracy_score(y_train,dt.predict(x_train))

0.9972303431297123

In [105]:
accuracy_score(y_test,dt.predict(x_test))

0.9963076923076923